In [1]:
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import spatialdata as sd
import scanpy as sc

In [2]:
adata = sc.read_h5ad('/storage/lingyuan2/STATES_data/mousebrain_0503.h5ad')

In [ ]:
adata

In [ ]:
adata

In [ ]:
df = adata.uns['remain_reads_info_new']
# Get list of detected genes from adata.var
detected_genes = adata.var[adata.var['detected']].index.tolist()

# Filter df to only include detected genes
df = df[df['gene'].isin(detected_genes)]
df

In [ ]:
# Group by gene and cell_barcode type (-1 vs non -1)
processes = df[df['cell_barcode'] == -1].groupby(['gene', 'type']).size().unstack(fill_value=0)
somata = df[df['cell_barcode'] != -1].groupby(['gene', 'type']).size().unstack(fill_value=0)

# Calculate rbRNA/(rbRNA+ntRNA) TE for each group
processes['TE'] = processes['rbRNA'] / (processes['rbRNA'] + processes['ntRNA'])
somata['TE'] = somata['rbRNA'] / (somata['rbRNA'] + somata['ntRNA'])

# Combine results into a single dataframe
result = pd.DataFrame({
    'processes_ntRNA': processes['ntRNA'],
    'processes_rbRNA': processes['rbRNA'], 
    'processes_TE': processes['TE'],
    'somata_ntRNA': somata['ntRNA'],
    'somata_rbRNA': somata['rbRNA'],
    'somata_TE': somata['TE']
})

result

In [ ]:
# Calculate TE difference between somata and processes
result['TE_diff'] = result['somata_TE'] - result['processes_TE']
result

In [ ]:
# Calculate processes percentage
result['processes%'] = (result['processes_ntRNA'] + result['processes_rbRNA']) / (
    result['processes_ntRNA'] + result['processes_rbRNA'] + 
    result['somata_ntRNA'] + result['somata_rbRNA']
)

result

In [ ]:
# Add rank columns for TE_diff and processes%
result['TE_diff_rank'] = result['TE_diff'].rank(ascending=False).astype(int)
result['processes%_rank'] = result['processes%'].rank(ascending=False).astype(int)
result

In [1]:
import pandas as pd

# 读入soma_process_eachgene_all_0503.csv
df = pd.read_csv('soma_process_eachgene_all_0503.csv', index_col=0)


In [ ]:
import numpy as np
df['fc'] = df['processes_TE'] / df['somata_TE']
df['log2fc'] = np.log2(df['fc'])
df

In [ ]:
# Add a column for somata_TE/processes_TE ratio
result['fc'] = result['somata_TE'] / result['processes_TE']
result['log2fc'] = np.log2(result['fc'])
result

In [6]:
df.to_csv('soma_process_eachgene_all_0503.csv')

In [9]:
import pandas as pd

df = pd.read_csv('soma_process_eachgene_all_0503.csv', index_col=0)

df_fdr = pd.read_csv('gene_TE_comparison_results_with_FDR_all0503.csv')

df_fdr = df_fdr.set_index('gene')

df['adj_pvalue'] = df_fdr.loc[df.index, 'adj_pvalue']
df['significant'] = df_fdr.loc[df.index, 'significant']

df.to_csv('eachgenefinal_all0503.csv')
